In [27]:
import numpy as np
import pandas as pd

a = 6378.137
b = 6356.752
f = 0.003352810664747481
e_sqr = 0.00669437999014131

#dataset for geographic latitudes
#before correction

df_8 = pd.DataFrame({
    "PHI_E" : [40, 60, 0],
    "LAMBDA_E" : [0, 0, -30],
    "PHI_S" : [50, 70, 60],
    "LAMBDA_S" : [10, 10, 60]
})
df_8

,PHI_E,LAMBDA_E,PHI_S,LAMBDA_S
0,40,0,50,10
1,60,0,70,10
2,0,-30,60,60


In [28]:
# latitudes conversion from geographic to geocentric

phi_e_c = np.degrees(np.arctan(((1 - e_sqr) * np.tan(np.radians(df_8["PHI_E"])))))
phi_s_c = np.degrees(np.arctan(((1 - e_sqr) * np.tan(np.radians(df_8["PHI_S"])))))

# creating new_dataset
df_8 = pd.DataFrame({
    "PHI_E_C" : phi_e_c,
    "LAMBDA_E" : [0, 0, -30],
    "PHI_S_C" : phi_s_c,
    "LAMBDA_S" : [10, 10, 60]
})

df_8

,PHI_E_C,LAMBDA_E,PHI_S_C,LAMBDA_S
0,39.810611,0,49.810390,10
1,59.833076,0,69.875993,10
2,0.000000,-30,59.833076,60


In [29]:
e1 = np.array(np.cos(np.radians(df_8["LAMBDA_E"]))*np.cos(np.radians(df_8["PHI_E_C"])))
e2 = np.array(np.sin(np.radians(df_8["LAMBDA_E"]))*np.cos(np.radians(df_8["PHI_E_C"])))
e3 = np.array(np.sin(np.radians(df_8["PHI_E_C"])))

s1 = np.array(np.cos(np.radians(df_8["LAMBDA_S"]))*np.cos(np.radians(df_8["PHI_S_C"])))
s2 = np.array(np.sin(np.radians(df_8["LAMBDA_S"]))*np.cos(np.radians(df_8["PHI_S_C"])))
s3 = np.array(np.sin(np.radians(df_8["PHI_S_C"])))

In [30]:
cos_d = e1*s1 + e2*s2 + e3*s3
sin_d_half = 0.5 * np.sqrt( np.power((s1 - e1),2) + np.power((s2 - e2),2) + np.power((s3 - e3),2))
cos_d_half = 0.5 * np.sqrt( np.power((s1 + e1),2) + np.power((s2 + e2),2) + np.power((s3 + e3),2))
sin_d = 0.5*2*2*sin_d_half * cos_d_half

delta = np.degrees(np.arctan(sin_d / cos_d))

print("delta for 1st row = {:.2f} deg".format(delta[0]))
print("delta for 2nd row = {:.2f} deg".format(delta[1]))
print("delta for 3rd row = {:.2f} deg".format(delta[2]))

delta for 1st row = 12.24 deg
delta for 2nd row = 10.87 deg
delta for 3rd row = 90.00 deg


## Azimuth calculation

$$ Az = atan2(\cos\phi_s \cos\phi_e \sin(\lambda_s - \lambda_e) , \sin\phi_s - \cos\Delta\sin\phi_e) $$

In [31]:
# Azimuth calculation

y = np.cos(np.radians(df_8["PHI_S_C"]))*np.cos(np.radians(df_8["PHI_E_C"]))* np.sin(
    np.radians(df_8["LAMBDA_S"]) - np.radians(df_8["LAMBDA_E"]))
x = np.sin(np.radians(df_8["PHI_S_C"])) - (cos_d* np.sin(np.radians(df_8["PHI_E_C"])))
azimuth = np.degrees(np.arctan(y/x))

print("Azimuth for the 1st row = {:.2f}".format(azimuth[0])+chr(176))
print("Azimuth for the 2nd row = {:.2f}".format(azimuth[1])+chr(176))
print("Azimuth for the 3rd row = {:.2f}".format(azimuth[2])+chr(176))

Azimuth for the 1st row = 31.92°
Azimuth for the 2nd row = 18.47°
Azimuth for the 3rd row = 30.17°


In [32]:
#backazimuth calculation

a = np.cos(np.radians(df_8["PHI_E_C"]))*np.cos(np.radians(df_8["PHI_S_C"]))*np.sin(
    np.radians(df_8["LAMBDA_E"]) - np.radians(df_8["LAMBDA_S"]))
b = np.sin(np.radians(df_8["PHI_E_C"])) - (cos_d* np.sin(np.radians(df_8["PHI_S_C"])))
baz = np.degrees(np.arctan(a/b)) + 180

print("Baz 1st = {:2f}".format(baz[0]) + chr(176))
print("Baz 2nd = {:2f}".format(baz[1]) + chr(176))
print("Baz 3rd = {:2f}".format(baz[2]) + chr(176))

Baz 1st = 218.998994°
Baz 2nd = 207.557647°
Baz 3rd = 270.000000°


## Dataset after correction into geocentric latitudes

In [33]:
# generating the entire dataset after converting into geocentrict latitudes

df_8["D(deg)"] = delta
df_8["Az(deg)"] = azimuth
df_8["Baz(deg)"] = baz
df_8

,PHI_E_C,LAMBDA_E,PHI_S_C,LAMBDA_S,D(deg),Az(deg),Baz(deg)
0,39.810611,0,49.810390,10,12.237479,31.915455,218.998994
1,59.833076,0,69.875993,10,10.872075,18.466435,207.557647
2,0.000000,-30,59.833076,60,90.000000,30.166924,270.000000


## The codes from assignment 7

In [34]:
# Codes from assignment 7 using geographic latitudes
dataset = pd.DataFrame({
    "PHI_E" : [40, 60, 0],
    "LAMBDA_E" : [0, 0, -30],
    "PHI_S" : [50, 70, 60],
    "LAMBDA_S" : [10, 10, 60]
})

e1 = np.array(np.cos(np.radians(dataset["LAMBDA_E"]))*np.cos(np.radians(dataset["PHI_E"])))
e2 = np.array(np.sin(np.radians(dataset["LAMBDA_E"]))*np.cos(np.radians(dataset["PHI_E"])))
e3 = np.array(np.sin(np.radians(dataset["PHI_E"])))

s1 = np.array(np.cos(np.radians(dataset["LAMBDA_S"]))*np.cos(np.radians(dataset["PHI_S"])))
s2 = np.array(np.sin(np.radians(dataset["LAMBDA_S"]))*np.cos(np.radians(dataset["PHI_S"])))
s3 = np.array(np.sin(np.radians(dataset["PHI_S"])))

cos_d = e1*s1 + e2*s2 + e3*s3
sin_d_half = 0.5 * np.sqrt( np.power((s1 - e1),2) + np.power((s2 - e2),2) + np.power((s3 - e3),2))
cos_d_half = 0.5 * np.sqrt( np.power((s1 + e1),2) + np.power((s2 + e2),2) + np.power((s3 + e3),2))
sin_d = 0.5*2*2*sin_d_half * cos_d_half
d = np.degrees(np.arctan(sin_d / cos_d))
#dist_km = d*111.195

# Azimuth calculation

y = np.cos(np.radians(dataset["PHI_S"]))*np.cos(np.radians(dataset["PHI_E"]))* np.sin(
    np.radians(dataset["LAMBDA_S"]) - np.radians(dataset["LAMBDA_E"]))
x = np.sin(np.radians(dataset["PHI_S"])) - (cos_d* np.sin(np.radians(dataset["PHI_E"])))
azimuth = np.degrees(np.arctan(y/x))

#backazimuth calculation

a = np.cos(np.radians(dataset["PHI_E"]))*np.cos(np.radians(dataset["PHI_S"]))*np.sin(
    np.radians(dataset["LAMBDA_E"]) - np.radians(dataset["LAMBDA_S"]))
b = np.sin(np.radians(dataset["PHI_E"])) - (cos_d* np.sin(np.radians(dataset["PHI_S"])))
baz = np.degrees(np.arctan(a/b)) + 180

## dataset without geocentric latitudes correction

In [35]:
# generating the entire dataset

dataset["D(deg)"] = d
dataset["Az(deg)"] = azimuth
dataset["Baz(deg)"] = baz
dataset

,PHI_E,LAMBDA_E,PHI_S,LAMBDA_S,D(deg),Az(deg),Baz(deg)
0,40,0,50,10,12.224070,31.813918,218.920943
1,60,0,70,10,10.823679,18.437403,207.539077
2,0,-30,60,60,90.000000,30.000000,270.000000


## Comparison between two dataset
## Comparison between distance(km) on 2 dataframes
## Comparison between them before and after correction into geocentric latitudes

In [36]:
# After correction into geocentric latitudes
df_8

,PHI_E_C,LAMBDA_E,PHI_S_C,LAMBDA_S,D(deg),Az(deg),Baz(deg)
0,39.810611,0,49.810390,10,12.237479,31.915455,218.998994
1,59.833076,0,69.875993,10,10.872075,18.466435,207.557647
2,0.000000,-30,59.833076,60,90.000000,30.166924,270.000000


In [37]:
# before correction into geocentric latitudes
dataset

,PHI_E,LAMBDA_E,PHI_S,LAMBDA_S,D(deg),Az(deg),Baz(deg)
0,40,0,50,10,12.224070,31.813918,218.920943
1,60,0,70,10,10.823679,18.437403,207.539077
2,0,-30,60,60,90.000000,30.000000,270.000000
